### Pip install

In [1]:
%%capture
!pip install markovify

In [2]:
%%capture
!pip install tweepy

### Import stuff

In [3]:
import csv
import markovify
import os
import re
import tweepy

### Config

In [4]:
consumer_key = os.environ['CONSUMER_KEY']
consumer_secret = os.environ['CONSUMER_SECRET']
access_key = os.environ['ACCESS_KEY']
access_secret = os.environ['ACCESS_SECRET']

### Lib code

In [5]:
def tweepyClientFactory():
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    
    return tweepy.API(auth)

In [6]:
def getAllTweets(screen_name):
    client = tweepyClientFactory()
    
    return [tweet.full_text for tweet in tweepy.Cursor(client.user_timeline, tweet_mode='extended', screen_name='realDonaldTrump').items(1000)]

In [7]:
def _cleanTweet(tweet):
    tweet = re.sub(r"http\S+", "", tweet)       #links
    tweet = re.sub("#\S+", "", tweet)           #hashtags
    tweet = re.sub("@[A-Za-z0-9]+", "", tweet) 
    tweet = re.sub("RT.+", "", tweet)           #Retweets
    tweet = re.sub("Video\:", "", tweet)        #Videos
    tweet = re.sub("\n", "", tweet)             #new lines
    tweet = re.sub("^\.\s.", "", tweet)         #leading whitespace
    tweet = re.sub("\s+", " ", tweet)           #extra whitespace
    tweet = re.sub("&amp;", "and", tweet)       #encoded ampersands
        
    return tweet.strip()

In [8]:
def cleanTweets(tweets):
    cleanedTweets = [_cleanTweet(tweet) for tweet in tweets]
    
    while '' in cleanedTweets:
        cleanedTweets.remove('')
    
    return cleanedTweets

In [9]:
def writeTweetsToCsv(tweets):
    with open('tweets.csv', 'w') as f:
        writer = csv.writer(f)
        for tweet in tweets:
            writer.writerow([tweet])

In [10]:
def getTweetsFromCsv():
    with open("tweets.csv") as f:
        return f.read()

### Impl

In [11]:
tweets = getAllTweets("realDonaldTrump")

In [12]:
cleanedTweets = cleanTweets(tweets)

In [13]:
writeTweetsToCsv(cleanedTweets)

In [14]:
tweetsFromCsv = getTweetsFromCsv()

In [15]:
model = markovify.NewlineText(tweetsFromCsv)

### Test

In [16]:
model.make_sentence()

'I am on my way to Singapore and talks with North Korea on Denuclearization. Won’t be talking about the phony Russian Witch Hunt.'